## Необходимые модули: 

conda create -n `EnvName` python=3.11

conda activate `EnvName`

conda install beautifoulsoup4

conda install requests

pip install fake-useragent

pip install zenrows

###### Есть некоторые несовместимости, между дальнейшими библиотеками(pytorch, ultralytics), поэтому они не должны пересекаться в одном окружении.

________________________

## 1. Парсим ссылки с platesmania

Изначально были трудности с Cloudflare по ссылкам: 
* https://platesmania.com/al/gallery.php?ctype=1
* https://platesmania.com/al/gallery.php?ctype=2
* https://platesmania.com/al/gallery.php?ctype=3
* https://platesmania.com/al/gallery.php?ctype=4

Решение было найдено путем удаления расширения файла, данные ссылки не имели защиты: 
* https://platesmania.com/al/gallery?ctype=1
* https://platesmania.com/al/gallery?ctype=2
* https://platesmania.com/al/gallery?ctype=3
* https://platesmania.com/al/gallery?ctype=4

Была замеченна аномалия на сайте в разделе ctype=1, после 100-ой страницы постоянно повторяется одна и таже машина.
* https://platesmania.com/al/gallery?ctype=1&start=100

In [1]:
import requests 
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import json
import re

ctypes = [1, 2, 3, 4]
pages = [99, 6, 26, 66]
base_url = 'https://platesmania.com'
main_links = [[]] * len(ctypes)
headers = {'User-Agent': UserAgent().chrome}
codes = []

for i in range(len(ctypes)):
    ctype_links = []
    for j in range(pages[i]):
        url = f'https://platesmania.com/al/gallery?&ctype={ctypes[i]}&start={j}'
        response = requests.get(url, headers=headers, timeout=2.5)
        codes.append(response.status_code)
        print(str(j) + '.', response.status_code)
        soap = BeautifulSoup(response.content, 'html.parser').find_all('div', class_='panel panel-grey')
        all_links = [base_url + item.find('a', href=re.compile('/al/nomer'))['href'] for item in soap]
        ctype_links += all_links
    main_links[i] = ctype_links

links_dict = {'ctype1': main_links[0],
                'ctype2': main_links[1],
                'ctype3': main_links[2],
                'ctype4': main_links[3]}

with open('links.json', 'w') as fp:
    json.dump(links_dict, fp)
    
print('Ссылки успешно сохранены в JSON файл!')


0. 200
1. 200
2. 200
3. 200
4. 200
5. 200
6. 200
7. 200
8. 200
9. 200
10. 200
11. 200
12. 200
13. 200
14. 200
15. 200
16. 200
17. 200
18. 200
19. 200
20. 200
21. 200
22. 200
23. 200
24. 200
25. 200
26. 200
27. 200
28. 200
29. 200
30. 200
31. 200
32. 200
33. 200
34. 200
35. 200
36. 200
37. 200
38. 200
39. 200
40. 200
41. 200
42. 200
43. 200
44. 200
45. 200
46. 200
47. 200
48. 200
49. 200
50. 200
51. 200
52. 200
53. 200
54. 200
55. 200
56. 200
57. 200
58. 200
59. 200
60. 200
61. 200
62. 200
63. 200
64. 200
65. 200
66. 200
67. 200
68. 200
69. 200
70. 200
71. 200
72. 200
73. 200
74. 200
75. 200
76. 200
77. 200
78. 200
79. 200
80. 200
81. 200
82. 200
83. 200
84. 200
85. 200
86. 200
87. 200
88. 200
89. 200
90. 200
91. 200
92. 200
93. 200
94. 200
95. 200
96. 200
97. 200
98. 200
0. 200
1. 200
2. 200
3. 200
4. 200
5. 200
0. 200
1. 200
2. 200
3. 200
4. 200
5. 200
6. 200
7. 200
8. 200
9. 200
10. 200
11. 200
12. 200
13. 200
14. 200
15. 200
16. 200
17. 200
18. 200
19. 200
20. 200
21. 200
22. 200
23

________________

## 2. Структура данных

* ads_link - ссылка на страницу номера
* link_text - п4 меты, обычно там хранится марка и модель машины
* main_img - ссылка на реальное изображение
* generated_img - ссылка на сгенерированный номер
* place_meta - п5 меты, обычно там хранится подробная информация об авто
* plate_number - п3 меты, там хранится расшифровка номера

______________

## 3. Скребем данные с карточек

Здесь появились аналогичные трудности с обходом защиты Cloudflare, но при этом было испробованно несколько способом, ни один из которых не стал решением:
* Подделка User-Agent`a и других опшенов
* Использование библиотек скрейперов
* Использование Selenium

Был выбран вариант пользоваться готовым АПИ от zenrows(АПИ платное). Так как АПИ оказалось плтаное, кроме автоматизации всей работы приходилось постоянно менять API keys, а также диапозон статей для парсинга.

In [51]:
from zenrows import ZenRowsClient

def read_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        return {}
    except json.JSONDecodeError:
        return {}

def write_json(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def parse_ctypes(ctype): 
    cards_data = {}
    count_cards = 0
    codes_for_cards = {}
    total_responses = 0 
    for link in links[ctype][976:]: # Смена диапозона происходила здесь
        print(link)
        if total_responses <= 40:
            count_responses = 0
            response = client.get(link, params=params)
            count_responses += 1
            status = response.status_code
            print(status)
            while status != 200 and count_responses <= 15:
                response = client.get(link, params=params)
                status = response.status_code
                print(status)
                count_responses += 1
                if status == 402:
                    print('Количество запросов кончилось!')
                    break
            total_responses += count_responses
            codes_for_cards[count_cards] = status
            count_cards += 1
            if status == 402:
                print('Количество запросов кончилось!')
                break
            elif status == 200:
                soap = BeautifulSoup(response.content, 'html.parser')
                cards_data['/' + '/'.join(link.split('/')[-2:])] = {
                    'ads_link': '/' + '/'.join(link.split('/')[-2:]),
                    'link_text': ' '.join([i.get_text() for i in soap.find('h3', class_='text-center margin-bottom-10').find_all('a')]) if soap.find('h3', class_='text-center margin-bottom-10') else '',
                    'main_img': soap.find('img', class_='img-responsive center-block')['src'] if soap.find('img', class_='img-responsive center-block') else '',
                    'generated_img': soap.find('img', class_='img-responsive center-block margin-bottom-20')['src'] if soap.find('img', class_='img-responsive center-block margin-bottom-20') else '',
                    'place_meta': soap.find('p', class_='text-center').find('a').get_text() if soap.find('p', class_='text-center') and soap.find('p', class_='text-center').find('a') else '',
                    'plate_number': soap.find('h1', class_='pull-left').get_text() if soap.find('h1', class_='pull-left') else ''
                }   
            else: 
                print(f'Не удалось получить доступ к карточке под номером {count_cards}: {status}')
        else:
            break
        print('Количество пройденных статей:', count_cards, '\nОбщее количество запросов:', total_responses)

    file_path = f'al-{ctype[:-1]}-{ctype[-1]}_meta.json'

    existing_data = read_json(file_path)

    existing_data.update(cards_data)

    write_json(file_path, existing_data)

    return (cards_data, count_cards, codes_for_cards)

client = ZenRowsClient("62e462ccb676a8f36c7c292a6982d6f95286f347") # Смена API key происходила здесь
params = {"js_render":"true", 
          "premium_proxy":"true"
          }

with open('links.json', 'r') as fp:
    links = json.load(fp)

In [52]:
info_ctype1 =  parse_ctypes('ctype1')

https://platesmania.com/al/nomer23155076
200
Количество пройденных статей: 1 
Общее количество запросов: 1
https://platesmania.com/al/nomer23155073
200
Количество пройденных статей: 2 
Общее количество запросов: 2
https://platesmania.com/al/nomer23155067
200
Количество пройденных статей: 3 
Общее количество запросов: 3
https://platesmania.com/al/nomer23155063
200
Количество пройденных статей: 4 
Общее количество запросов: 4
https://platesmania.com/al/nomer23155061
200
Количество пройденных статей: 5 
Общее количество запросов: 5
https://platesmania.com/al/nomer23155058
200
Количество пройденных статей: 6 
Общее количество запросов: 6
https://platesmania.com/al/nomer23155054
200
Количество пройденных статей: 7 
Общее количество запросов: 7
https://platesmania.com/al/nomer23155034
200
Количество пройденных статей: 8 
Общее количество запросов: 8
https://platesmania.com/al/nomer23155032
200
Количество пройденных статей: 9 
Общее количество запросов: 9
https://platesmania.com/al/nomer23155

In [9]:
info_ctype2 = parse_ctypes('ctype2')

https://platesmania.com/al/nomer17534297
200
Количество пройденных статей: 1 
Общее количество запросов: 1
https://platesmania.com/al/nomer17507096
200
Количество пройденных статей: 2 
Общее количество запросов: 2
https://platesmania.com/al/nomer17094215
200
Количество пройденных статей: 3 
Общее количество запросов: 3
https://platesmania.com/al/nomer16800484
200
Количество пройденных статей: 4 
Общее количество запросов: 4
https://platesmania.com/al/nomer16670928
200
Количество пройденных статей: 5 
Общее количество запросов: 5
https://platesmania.com/al/nomer16670925
200
Количество пройденных статей: 6 
Общее количество запросов: 6
https://platesmania.com/al/nomer16661448
200
Количество пройденных статей: 7 
Общее количество запросов: 7
https://platesmania.com/al/nomer16654778
200
Количество пройденных статей: 8 
Общее количество запросов: 8
https://platesmania.com/al/nomer16619263
200
Количество пройденных статей: 9 
Общее количество запросов: 9
https://platesmania.com/al/nomer15959

In [18]:
info_ctype3 = parse_ctypes('ctype3')

https://platesmania.com/al/nomer16936074
200
Количество пройденных статей: 1 
Общее количество запросов: 1
https://platesmania.com/al/nomer16892392
200
Количество пройденных статей: 2 
Общее количество запросов: 2
https://platesmania.com/al/nomer16874579
200
Количество пройденных статей: 3 
Общее количество запросов: 3
https://platesmania.com/al/nomer16874574
200
Количество пройденных статей: 4 
Общее количество запросов: 4
https://platesmania.com/al/nomer16866858
200
Количество пройденных статей: 5 
Общее количество запросов: 5
https://platesmania.com/al/nomer16866855
200
Количество пройденных статей: 6 
Общее количество запросов: 6
https://platesmania.com/al/nomer16777525
200
Количество пройденных статей: 7 
Общее количество запросов: 7
https://platesmania.com/al/nomer16638811
200
Количество пройденных статей: 8 
Общее количество запросов: 8
https://platesmania.com/al/nomer16543097
200
Количество пройденных статей: 9 
Общее количество запросов: 9
https://platesmania.com/al/nomer16299

In [77]:
info_ctype4 = parse_ctypes('ctype4')

https://platesmania.com/al/nomer15592955
200
Количество пройденных статей: 1 
Общее количество запросов: 1
https://platesmania.com/al/nomer15592922
200
Количество пройденных статей: 2 
Общее количество запросов: 2
https://platesmania.com/al/nomer15592496
200
Количество пройденных статей: 3 
Общее количество запросов: 3
https://platesmania.com/al/nomer15592402
200
Количество пройденных статей: 4 
Общее количество запросов: 4
https://platesmania.com/al/nomer15592361
200
Количество пройденных статей: 5 
Общее количество запросов: 5
https://platesmania.com/al/nomer15592354
200
Количество пройденных статей: 6 
Общее количество запросов: 6


__________________

## 4. Качаем фото и распределяем их

In [73]:
import os
import shutil

source_dir = r'C:\Users\aleks\Desktop\TestTask'
solution_dir = 'solution'

dirs = ['al-ctype-1', 'al-ctype-2', 'al-ctype-3', 'al-ctype-4']

for dir in dirs:
    path = os.path.join(solution_dir, dir)
    if not os.path.exists(path):
        os.makedirs(path)

files = ['al-ctype-1_meta.json', 'al-ctype-2_meta.json', 'al-ctype-3_meta.json', 'al-ctype-4_meta.json']

errors = []

for i in range(len(files)):
    if not os.path.exists(os.path.join(solution_dir, files[i])):
        shutil.move(os.path.join(files[i]), solution_dir)
    with open(os.path.join(solution_dir, files[i]), 'r') as f:
        data = json.load(f)
        for obj in data:
            path = os.path.join(solution_dir, dirs[i], data[obj]['ads_link'].replace('/', '_'))
            if not os.path.exists(path):
                os.makedirs(path)
            response_image = requests.get(data[obj]['main_img'], headers=headers, timeout=2.5)
            response_generated = requests.get(data[obj]['generated_img'], headers=headers, timeout=2.5)
            if response_image.status_code != 200 or response_generated.status_code != 200:
                errors.append(data[obj]['ads_link'].replace('/', '_'))
            else:
                with open(path + '\\generated_' + data[obj]['generated_img'].split('/')[-1], 'wb') as generated:
                    generated.write(response_generated.content)
                with open(path + '\\real_' + data[obj]['main_img'].split('/')[-1], 'wb') as generated:
                    generated.write(response_image.content)

if not errors:
    print('Все фото успешно сохранены в нужные дерриктории!')
else:
    for error in errors:
        print(f'Возникли ошибки в папке: {error}')

Все фото успешно сохранены в нужные дерриктории!


Данные успешно собраны и структурированны!

______________________________